## NLP
### Pré processamento de texto
<b>Objetivo: </b> Entender porque preprocessamentos são importantes para tarefas envolvendo texto e como limpar o texto.
<br><b>Autora:</b> Renata Gotler

Ao longo da disciplina de NLP (Natural Language Preprocessing), estaremos usando os dados disponibilizados da americanas. Esse dataset contém as informações das avaliações dos produtos, como comentário, número de estrelas e mais. 
<br> Nessa primeira aula, vamos explorar o texto das avaliações, entender o porque a limpeza dos dados é importante e como realizá-la. Dessa forma, nessa aula focaremos nos comentários sobre os produtos.

In [1]:
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
url = 'https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/main/B2W-Reviews01.csv'

df_reviews = pd.read_csv(url)

/tmp/ipykernel_6731/3631190906.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv(url)


In [4]:
df_reviews.head()

,submission_date,reviewer_id,product_id,product_name,product_brand,site_category_lv1,site_category_lv2,review_title,overall_rating,recommend_to_a_friend,review_text,reviewer_birth_year,reviewer_gender,reviewer_state
0,2018-01-01 00:11:28,d0fb1ca69422530334178f5c8624aa7a99da47907c44de...,132532965,Notebook Asus Vivobook Max X541NA-GO472T Intel...,NaN,Informática,Notebook,Bom,4,Yes,Estou contente com a compra entrega rápida o ú...,1958.0,F,RJ
1,2018-01-01 00:13:48,014d6dc5a10aed1ff1e6f349fb2b059a2d3de511c7538a...,22562178,Copo Acrílico Com Canudo 500ml Rocie,NaN,Utilidades Domésticas,"Copos, Taças e Canecas","Preço imbatível, ótima qualidade",4,Yes,"Por apenas R$1994.20,eu consegui comprar esse ...",1996.0,M,SC
2,2018-01-01 00:26:02,44f2c8edd93471926fff601274b8b2b5c4824e386ae4f2...,113022329,Panela de Pressão Elétrica Philips Walita Dail...,philips walita,Eletroportáteis,Panela Elétrica,ATENDE TODAS AS EXPECTATIVA.,4,Yes,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,1984.0,M,SP
3,2018-01-01 00:35:54,ce741665c1764ab2d77539e18d0e4f66dde6213c9f0863...,113851581,Betoneira Columbus - Roma Brinquedos,roma jensen,Brinquedos,Veículos de Brinquedo,presente mais que desejado,4,Yes,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,1985.0,F,SP
4,2018-01-01 01:00:28,7d7b6b18dda804a897359276cef0ca252f9932bf4b5c8e...,131788803,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",lg,TV e Home Theater,TV,"Sem duvidas, excelente",5,Yes,"A entrega foi no prazo, as americanas estão de...",1994.0,M,MG


Abaixo podemos ver que:
- Temos um total de 132.373 avaliações. No entanto, review_text possui nulos, vamos começar tirando essas avaliações sem texto, visto que não podemos classificar sentimento de um texto inexistente. 
- Overall rating já está como inteiro, então não precisamos transformar essa coluna.

In [5]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132373 entries, 0 to 132372
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   submission_date        132373 non-null  object 
 1   reviewer_id            132373 non-null  object 
 2   product_id             132373 non-null  object 
 3   product_name           132289 non-null  object 
 4   product_brand          40982 non-null   object 
 5   site_category_lv1      132367 non-null  object 
 6   site_category_lv2      128360 non-null  object 
 7   review_title           132071 non-null  object 
 8   overall_rating         132373 non-null  int64  
 9   recommend_to_a_friend  132355 non-null  object 
 10  review_text            129098 non-null  object 
 11  reviewer_birth_year    126389 non-null  float64
 12  reviewer_gender        128237 non-null  object 
 13  reviewer_state         128382 non-null  object 
dtypes: float64(1), int64(1), object(12)


In [6]:
df_reviews = df_reviews[["review_text", "overall_rating"]].dropna()

Abaixo já conseguimos ter uma ideia do que nos espera, mistura de letras maiúsculas com minúsculas, cada pessoa com um estilo de escrita diferente. O que mais podemos esperar? 

In [7]:
df_reviews.head()

,review_text,overall_rating
0,Estou contente com a compra entrega rápida o ú...,4
1,"Por apenas R$1994.20,eu consegui comprar esse ...",4
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,4
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,4
4,"A entrega foi no prazo, as americanas estão de...",5


Os algoritmos de machine learning entendem somente números, não texto. Dessa forma, precisamos criar representações vetoriais numéricas para que o algoritmo aprende padrões e consiga fazer inferências sobre dados novos. 
<br>O primeiro passo para isso é quebrarmos o texto em unidades menores, chamadas tokens. Podemos realizar essa operação de várias formas: por caracter, palavra ou subpalavra, por exemplo.
<br><b> O principal objetivo da tokenização é criar um vocabulário mínimo que reduza a quantidade de palavras desconhecidas.</b>

#### Tokenização
Vamos começar um exercício, quebrando o nosso texto de diversas formas diferentes e comparando o vocabulário que teríamos.

In [8]:
import nltk
import itertools

from transformers import BertTokenizer
from nltk import word_tokenize

from src.utils import count_tokens

nltk.download("punkt")
nltk.download("rslp")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
count_vocab = {}

##### Tokenização por caracter

Podemos fazer a separação mais simples de todas: por caracter, isso reduziria bastante nosso vocabulário, certo? Temos um máximo que podemos atingir, que é a quantidade de letras no alfabeto * 2 (maiúscula e minúscula) + variações de acentuação + pontuação + caracteres especiais, mas não vai tão além disso... 

In [10]:
df_reviews["token_caracter"] = df_reviews["review_text"].apply(lambda x: list(x))
df_reviews["token_caracter"]

0         [E, s, t, o, u,  , c, o, n, t, e, n, t, e,  , ...
1         [P, o, r,  , a, p, e, n, a, s,  , R, $, 1, 9, ...
2         [S, U, P, E, R, A,  , E, M,  , A, G, I, L, I, ...
3         [M, E, U,  , F, I, L, H, O,  , A, M, O, U, !, ...
4         [A,  , e, n, t, r, e, g, a,  , f, o, i,  , n, ...
                                ...                        
132368    [V, a, l, e,  , m, u, i, t, o, ,,  , e, s, t, ...
132369    [P, r, á, t, i, c, o,  , e,  , b, a, r, a, t, ...
132370    [C, h, e, g, o, u,  , a, n, t, e, s,  , d, o, ...
132371    [M, a, t, e, r, i, a, l,  , f, r, a, c, o, ,, ...
132372    [C, o, m, p, r, e, i,  , e, s, s, e,  , p, r, ...
Name: token_caracter, Length: 129098, dtype: object

Podemos ver que o espaço foi o caracter com maior frequência, seguido por vogais, as menos frequentes foram símbolos, o que faz bastante sentido. No total, nosso vocabulário ficou com tamanho 212.

In [11]:
caracter_counter = count_tokens(df=df_reviews, token_col="token_caracter")
count_vocab["caracter_counter"] = len(caracter_counter)
caracter_counter.most_common(5)

Tamanho do vocabulário total: 212


[(' ', 2892807), ('e', 1638340), ('o', 1554108), ('a', 1504508), ('r', 891104)]

In [12]:
caracter_counter.most_common()[-5:]

[('{', 1), ('}', 1), ('͜', 1), ('ʖ', 1), ('ķ', 1)]

Vamos olhar esses caracteres especiais? Aparentemente aqule caracter especial ķ foi um erro de digitação no meio dos k que representam uma risada, enquanto o ʖ é o nariz de um rosto. Começamos a perceber melhor porque limpar os dados, não é mesmo?

In [13]:
df_reviews[df_reviews["review_text"].str.contains('ķ')]["review_text"].iloc[0]

'Ótimo produto.. Indico para todos! Muito show.  Kkkkkķkkkkk'

In [14]:
df_reviews[df_reviews["review_text"].str.contains('ʖ')]["review_text"].iloc[0]

'Gostei muito, sempre que estou em perigo Tony vem me salvar com aquela linda musica do AC/DC. Recomendo muito este produto. Aquela armadura faz horrores gente ( ͡° ͜ʖ ͡°)'

É um excelente número de vocabulário, no entanto você já deve saber o problema em dividir o texto dessa forma: é difícil de extrair padrões, relações entre as palavras, porque primeiro o algoritmo vai ter que aprender a "remontar" a palavra. A letra por si só não trás informação, com isso, perdemos muita informação. É por isso que essa forma de tokenização não é muito usada na prática.

##### Tokenização por palavra

Uma forma bastante usada de tokenização é por palavra, conseguimos extrair padrões do texto através delas. Vamos ver na prática?

In [15]:
df_reviews["token_word"] = df_reviews["review_text"].apply(lambda x: word_tokenize(x))
df_reviews["token_word"]

0         [Estou, contente, com, a, compra, entrega, ráp...
1         [Por, apenas, R, $, 1994.20, ,, eu, consegui, ...
2         [SUPERA, EM, AGILIDADE, E, PRATICIDADE, OUTRAS...
3         [MEU, FILHO, AMOU, !, PARECE, DE, VERDADE, COM...
4         [A, entrega, foi, no, prazo, ,, as, americanas...
                                ...                        
132368    [Vale, muito, ,, estou, usando, no, controle, ...
132369    [Prático, e, barato, ,, super, indico, o, prod...
132370    [Chegou, antes, do, prazo, previsto, e, corres...
132371    [Material, fraco, ,, poderia, ser, melhor, ., ...
132372    [Comprei, esse, produto, ,, quando, chegou, es...
Name: token_word, Length: 129098, dtype: object

Podemos ver abaixo que nosso vocabulário aumentou significativamente, certo? De 212 fomos para 75.717! Cada variação teve seu próprio token, temos um token para A, outro para a, outro para as, outro para AS, e assim por diante, mas todos esses tokens tem o mesmo significado, podemos reduzir nosso vocabulário.

In [16]:
word_counter = count_tokens(df=df_reviews, token_col="token_word")
count_vocab["word_counter"] = len(word_counter)
word_counter.most_common(5)

Tamanho do vocabulário total: 75717


[(',', 175846), ('.', 175179), ('e', 95567), ('o', 82453), ('de', 80880)]

In [17]:
word_counter.most_common()[-5:]

[('NY', 1), ('largam', 1), ('21kilis', 1), ('1metro', 1), ('fortinha', 1)]

##### Tokenização por subpalavra

E se pudessemos fazer um intermediário entre a tokenização por caracter e por palavra? Podemos reduzir o vocabulário sem perder tanta informação usando a tokenização por subpalavra, que é bastante utilizada em modelos pré-treinados mais complexos, como vamos ver nas próximas aulas. Nessa aula vamos mostrar como exemplo a tokenização por subpalavra usada no Bert, que é a WordPiece.

In [18]:
bert_tokenizer = BertTokenizer.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    do_lower_case=True,
    cache_dir="../models/"
)

In [19]:
df_reviews["token_subword"] = df_reviews["review_text"].apply(lambda x: bert_tokenizer.tokenize(x))
df_reviews["token_subword"]

0         [estou, conte, ##nte, com, a, compra, entrega,...
1         [por, apenas, r, $, 1994, ., 20, ,, eu, conseg...
2         [supera, em, ag, ##ilidade, e, pratic, ##idade...
3         [meu, filho, amo, ##u, !, parece, de, verdade,...
4         [a, entrega, foi, no, prazo, ,, as, americanas...
                                ...                        
132368    [vale, muito, ,, estou, usando, no, controle, ...
132369    [pratic, ##o, e, bara, ##to, ,, super, indic, ...
132370    [chegou, antes, do, prazo, previsto, e, corres...
132371    [material, fraco, ,, poderia, ser, melhor, ., ...
132372    [compre, ##i, esse, produto, ,, quando, chegou...
Name: token_subword, Length: 129098, dtype: object

11259 tokens! Muito melhor né?
<br> Percebam que abaixo já fizemos uma limpeza muito comum, que é de transformar o texto para letra minúscula, o que já homogeniza bastante os dados.
<br>Percebam também que abaixo vários tokens puderam ser agrupados usando o método da subpalavra.
<br>Exemplo: "produto" foi quebrado em um token, enquanto temos outro token para "##s". Dessa forma, mesmo uma palavra no plural ou no singular, conseguimos agrupá-la, tratando com um token o que representa a mesma coisa, e outro que contém a informação se é plural ou singular.
<br>O mesmo acontece com vários outros tokens que conterão informações diferentes, como masculino e feminino, verbos e outros. Acima conseguimos ver um exemplo de divisão de verbo, onde um token foi dedicado a "compre" e outro para "##i", e um para "amo" e outro para "##u". As subpalavras sempre terão os ## no início, um padrão do WordPiece, que ajuda depois a voltarmos para a palavra original.

In [20]:
subword_counter = count_tokens(df=df_reviews, token_col="token_subword")
count_vocab["subword_counter"] = len(subword_counter)
subword_counter.most_common(5)

Tamanho do vocabulário total: 11259


[('.', 344998), ('##o', 179916), (',', 177398), ('e', 151051), ('o', 104337)]

In [21]:
subword_counter.most_common()[-5:]

[('pertencia', 1), ('##gaz', 1), ('hec', 1), ('ergue', 1), ('regressa', 1)]

As subpalavras ajudam bastante porque conseguem manter informação ao mesmo tempo que agrupam de forma eficiente os tokens, reduzindo o vocabulário.

#### Técnicas de normalização ou homogenização de texto
Para fins didáticos, vamos seguir olhando a contagem de vocabulário por palavra para demonstrar a diferença das transformações aplicadas. Percebam que ao mesmo tempo que vamos limpando o texto, também é possível que a transformação faça com que percamos parte de informação, que pode ou não ser relevante dependendo do problema e a forma que formos tratar ele.

In [22]:
import re
import spacy

from src.preprocessing import TextPreprocessing

prep = TextPreprocessing()

##### Transformar o texto para letra minúscula

Normalizar o texto para minúsculo faz com que tokens sejam agrupados independente de teremos letras maiúscula ou minúscula. Essa é uma técnica muito utilizada e funciona bem para grande parte dos casos. <br> Você consegue imaginar quando é importante essa distinção? Podemos pensar em pessoas que usam as letras maiúsculas para expressar um sentimento ruim, quando estão "gritando", ou em problemas de reconhecimento de entidades nomeadas, organizações e nomes próprios geralmente começam com maiúscula.

In [23]:
df_reviews["cleaned_text"] = df_reviews["review_text"].apply(lambda x: x.lower())
df_reviews["cleaned_text_tokenized"] = df_reviews["cleaned_text"].apply(lambda x: word_tokenize(x))
lower_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_tokenized")
count_vocab["lower_word_count"] = len(lower_word_count)

Tamanho do vocabulário total: 59363


Reduzimos bastante nosso vocabulário, né? de 75k para 60k.

##### Remover pontuação
Remover pontuação também nos ajuda a reduzir o vocabulário. Que nem comentamos, as limpezas também fazem com que percamos informação, aqui podemos perder informação sobre separação de sentença, vários ! podem indicar um sentimento ruim, dependendo do problema e da modelagem escolhida, a pontuação pode ou não ser necessária.
<br> Vamos continuar usando nossa coluna limpa para entendermos o impacto dessas limpezas no final.

Abaixo vemos que aumentamos nosso vocabulário ao invés de reduzirmos, o que não é o que queremos. Vamos entender porque?

In [24]:
df_reviews["cleaned_text_tokenized_without_ponctuation"] = df_reviews["cleaned_text"].apply(lambda x: word_tokenize(re.sub(r"[^\w\s]", "", x)))
lower_without_ponctuation_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_tokenized_without_ponctuation")
count_vocab["lower_without_ponctuation_word_count"] = len(lower_without_ponctuation_word_count)

Tamanho do vocabulário total: 66617


Abaixo podemos ver que tinhamos algumas palavras que eram separadas por causa de alguma pontuação. Quando retiramos a pontuação, as palavras se uniram, o que aumentou a variabilidade, aumentando a quantidade de tokens.

In [25]:
set(itertools.islice((lower_without_ponctuation_word_count.keys() - lower_word_count.keys()), 5))

{'0207', '1490', 'bemsilencioso', 'defeitosvamos', 'dissoestou'}

E se ao invés de retirar, substituíssemos a pontuação por um espaço? É assim que precisamos formular nosso raciocínio, criando hipótese, pensando em possibilidades de melhoria, e ir iterando para ver se estamos atingindo o resultado esperado. Agora nossa hipótese é: se eu substituir a pontuação por espaço, eu reduzo a quantidade de vocabulário.
<br> Nossa hipótese foi confirmada, de 59.363 para 50.393 tokens. Estamos conseguindo agora nosso objetivo de reduzir a quantidade de vocabulário.

In [26]:
df_reviews["cleaned_text"] = df_reviews["cleaned_text"].apply(lambda x: re.sub(r"[^\w\s]", " ", x))
df_reviews["cleaned_text_tokenized"] = df_reviews["cleaned_text"].apply(lambda x: word_tokenize(x))
lower_without_ponctuation_space_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_tokenized")
count_vocab["lower_without_ponctuation_space_word_count"] = len(lower_without_ponctuation_space_word_count)

Tamanho do vocabulário total: 50393


##### Padronizar para ASCII

Agora vamos testar aplicar o unidecode, que transformar todo o texto para um padrão chamado ASCII (American Standard Code for Information Interchange). Isso ajudará a tratar acentuação, aqueles caracteres especiais que vimos no início e mais.

In [27]:
df_reviews["cleaned_text"] = df_reviews["cleaned_text"].apply(lambda x: prep.preprocess_text(text=x, apply_unidecode=True))
df_reviews["cleaned_text_tokenized"] = df_reviews["cleaned_text"].apply(lambda x: word_tokenize(x))
unidecode_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_tokenized")
count_vocab["unidecode_word_count"] = len(unidecode_word_count)

Tamanho do vocabulário total: 47715


Vamos ver se conseguimos limpar alguns dos casos de símbolos que vimos?

In [28]:
df_reviews[df_reviews["review_text"].str.contains('ķ')]["cleaned_text"].iloc[0]

'otimo produto   indico para todos  muito show   kkkkkkkkkkk'

In [29]:
df_reviews[df_reviews["review_text"].str.contains('ʖ')]["cleaned_text"].iloc[0]

'gostei muito  sempre que estou em perigo tony vem me salvar com aquela linda musica do ac dc  recomendo muito este produto  aquela armadura faz horrores gente       ?    '

Excelente! Ambos os símbolos foram tratados, aquele ķ virou um k normal, enquanto o ʖ foi transformado em ?. Além disso, já temos um vocabulário de 47.715 tokens, bem menor do que o que iniciamos de 75.717, né?

##### Remoção de Stopwords
Quando lidamos principalmente com modelagems de algoritmos tradicionais que veremos nas próximas aulas, as palavras muito frequentes, chamadas de stopwords, como preposições, não ajudam na diferenciação de classes. Dessa forma, elas mais atrapalham a inferência na maioria das vezes do que ajudam, o que mais vai ajudar o algoritmo a diferenciar classes em uma classificação por exemplo são as palavras distintar mais específicas de uma determinada classe. 
<br>Podemos reduzir o vocabulário e melhorar a qualidade do modelo removendo essas palavras frequentes.

A biblioteca nltk já possui uma lista de stopwords de diversos idiomas, como o português, abaixo vamos olhar quais são as palavras já mapeadas.

In [30]:
print(f"Quantidade de stopwords mapeadas: {len(prep.get_stopwords())}")
prep.get_stopwords()[:10]

Quantidade de stopwords mapeadas: 207


['a',
 'à',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as']

Podemos usar essa biblioteca como ponto de partida e ir adicionando ou removendo palavras quando acharmos relevante para nosso problema.

In [31]:
df_reviews["cleaned_text"] = df_reviews["cleaned_text"].apply(lambda x: prep.preprocess_text(text=x, remove_stopwords=True))
df_reviews["cleaned_text_tokenized"] = df_reviews["cleaned_text"].apply(lambda x: word_tokenize(x))
stopword_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_tokenized")
count_vocab["stopword_word_count"] = len(stopword_word_count)

Tamanho do vocabulário total: 47563


Reduzimos pouco o vocabulário, até porque só temos 207 stopwords, mas isso, dependendo do problema em questão, pode fazer a diferença! 

##### Stemming

In [32]:
df_reviews["cleaned_text_stemming"] = df_reviews["cleaned_text"].apply(lambda x: prep.preprocess_text(text=x, apply_stemming=True))
df_reviews["cleaned_text_stemming_tokenized"] = df_reviews["cleaned_text_stemming"].apply(lambda x: word_tokenize(x))
stemming_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_stemming_tokenized")
count_vocab["stemming_word_count"] = len(stemming_word_count)

Tamanho do vocabulário total: 25620


Stemming é o processo de reduzir as palavras às suas raízes ou radicais. Percebam abaixo que esse método pode resultar em palavras não reconhecíveis, removendo sufixos e prefixos. Dessa forma, content foi reduzido para cont, compra foi reduzido para compr e assim por diante. Esse método consegue reduzir bastante o vocabulário, veja que chegamos a 25.620, 33% do vocabulário que iniciamos!

In [33]:
df_reviews[["review_text", "cleaned_text_stemming"]].iloc[:5]

,review_text,cleaned_text_stemming
0,Estou contente com a compra entrega rápida o ú...,cont compr entreg rap unic problem americ troc...
1,"Por apenas R$1994.20,eu consegui comprar esse ...",apen r 1994 20 consegu compr lind cop acril
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,sup agil pratic outr panel eletr costum us out...
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,filh amou parec verdad tant detalh
4,"A entrega foi no prazo, as americanas estão de...",entreg praz americ esta parab smart tv boa nav...


##### Lemmatizing
Reduz as palavras à sua forma canônica ou dicionária, o que geralmente produz resultados mais precisos e legíveis do que apenas cortar sufixos e prefixos como no stemming, além de manter sua integridade semântica.

In [34]:
lemmatizer = spacy.load('pt_core_news_sm')

df_reviews["cleaned_text_lemmatizing"] = df_reviews["cleaned_text"].apply(lambda x: prep.preprocess_text(text=x, apply_lemmitization=True, lemmatizer=lemmatizer))
df_reviews["cleaned_text_lemmatizing_tokenized"] = df_reviews["cleaned_text_lemmatizing"].apply(lambda x: word_tokenize(x))
lemmatizing_word_count = count_tokens(df=df_reviews, token_col="cleaned_text_lemmatizing_tokenized")
count_vocab["lemmatizing_word_count"] = len(lemmatizing_word_count)

Tamanho do vocabulário total: 41976


Abaixo podemos ver que os verbos foram normalizados, compra virou comprar, consegui virou conseguir e assim por diante. Além disso, outras foi normalizado para outro, tratamento de plurais acontecem na lemmatização, reduzindo nosso vocabulário. No entanto, podemos ver que não reduzimos tanto quanto usando o método stemming, veja que chegamos a 41.976 tokens.

In [35]:
df_reviews[["review_text", "cleaned_text_lemmatizing"]].iloc[:5]

,review_text,cleaned_text_lemmatizing
0,Estou contente com a compra entrega rápida o ú...,contente compra entregar raper unico problema ...
1,"Por apenas R$1994.20,eu consegui comprar esse ...",apenas r 1994 20 conseguir comprar lir copo ac...
2,SUPERA EM AGILIDADE E PRATICIDADE OUTRAS PANEL...,superar agilidade praticidade outro panela ele...
3,MEU FILHO AMOU! PARECE DE VERDADE COM TANTOS D...,filho amar parecer verdade tanto detalhe
4,"A entrega foi no prazo, as americanas estão de...",entregar prazo americana estao parabem smart t...


Abaixo podemos ver nossa evolução a medida que fomos limpando nosso texto, saindo de 75.717 e chegando ao mínimo de 25.620. Interessante notar que a tokenzização por subpalavra teve um vocabulário ainda menor (11.259) do que com todas essas limpezas, e perde muito menos informação. Interessante, não? Por isso esses métodos de tokenização são bastante usados, principalmente em algoritmos complexos como LLMs, e considerados o estado da arte.

In [36]:
count_vocab

{'caracter_counter': 212,
 'word_counter': 75717,
 'subword_counter': 11259,
 'lower_word_count': 59363,
 'lower_without_ponctuation_word_count': 66617,
 'lower_without_ponctuation_space_word_count': 50393,
 'unidecode_word_count': 47715,
 'stopword_word_count': 47563,
 'stemming_word_count': 25620,
 'lemmatizing_word_count': 41976}

Mesmo com todos as limpezas que fizemos, ainda temos um volume de vocabulário bem alto, podemos:
- Continuar com as limpezas, tratando urls, removendo números, removendo tokens com menos de 2 caracteres ou outras formas, dependendo do contexto e do problema;
- Definir um vocabulário máximo, removendo ou substituindo os tokens menos frequentes por "desconhecido" até chegarmos no tamanho de vocabulário definido;
- Definir um vocabulário, substituir os tokens menos frequentes pelos tokens mais frequentes usando esse vocabulário e métodos de similiridade de texto;
<br>Entre outros possíveis tratamentos.

#### Similaridade de texto (Correspondência Fuzzy)
Para corrigir erros de digitação, variações ortográficas, abreviações ou outros tipos de variações, podemos usar técnicas de correspondência fuzzy. Ela permite encontrar correspondências aproximadas entre textos, levando em consideração a semelhança entre eles, mesmo que não sejam idênticos. 
<br> Alguns exemplos de técnicas para correspondência fuzzy são:
- <b>Similaridade de Jaro:</b> Valor de 0 a 1, onde 1 é correspondência perfeita e 0 nenhuma correspondência. Basicamente é uma soma de correlações entre caracteres dos dois textos, ponderado pelo tamanho deles. 

$d_j=\frac{m}{3a}+\frac{m}{3b}+\frac{m-t}{3m}$

<br>onde:
- m é o número de correlações entre caracteres;
- a e b são os tamanhos dos textos a serem comparados;
- t é o número de transposições.


- <b>Similaridade de Jaro-Winkler:</b> Estende a Similaridade de Jaro adicionando uma constante que favorece quando a comparação ocorre com textos que possuem equivalência longa no início. </b>
- <b>Distância de Levenshtein:</b> Conta a quantidade mínima de operações que são necessárias para transformar um texto em outro. </b>
- <b>Distância de Damerau-Levenshtein:</b> Estende a distância de Levenshtein ao adicionar a transposição adjacente entre as possíveis operações. </b>

Vamos ver na prática como funciona? Vamos começar delimitando nosso máximo de vocabulário para 95% do que temos até a limpeza da removação das stopwords que foi a última limpeza antes do stemming e do lemmatizing, visto que elas alteram/cortam as palavras. Aqui 95% foi um número arbitrário, de acordo com o problema em questão, podemos aumentar ou reduzir. Dessa forma, estamos dizendo que 95% dos tokens são válidos, e os outros 5% vamos tentar tratá-los.
<br> Vamos começar com um exemplo de correção ortográfica que vamos retirar do nosso vocabulário de tratamento.

In [37]:
len(stopword_word_count)

47563

In [38]:
max_vocab = int(len(stopword_word_count) * 0.95)
items = stopword_word_count.most_common()
vocab_valido = dict(items[:max_vocab])
vocab_tratamento = dict(items[max_vocab:])

In [39]:
len(vocab_valido), len(vocab_tratamento)

(45184, 2379)

In [52]:
dict(items[-10:])

{'revigoram': 1,
 'stevia': 1,
 'porqua': 1,
 'schulz': 1,
 'regressao': 1,
 'ny': 1,
 'largam': 1,
 '21kilis': 1,
 '1metro': 1,
 'fortinha': 1}

Encontramos! `porqua` deveria ser `porque`. Vamos ver se conseguimos tratá-lo?

In [58]:
df_reviews[df_reviews["cleaned_text"].str.contains("porqua")]["review_text"].iloc[0]

'aonde eu encontro carvão de motor desta maquina, porqua a da schulz eu não estou encontrando.'

Abaixo vamos testar os métodos para tentar corrigir a palavra `porqua` para `porque`. Percebam que os métodos de Jaro são quanto maior mais similares são os textos, enquanto os métodos com levenshtein são quanto menor melhor, visto que precisamos de menos operações para chegar de um texto ao outro.

In [60]:
import jellyfish
import operator

similaridades = [
 "jaro_similarity",
 "jaro_winkler_similarity",
 "damerau_levenshtein_distance",
 "levenshtein_distance"
]

texto = "porqua"
resultados = {}
for distancia in similaridades:
  score = {
      valid_token: getattr(jellyfish, distancia)(
        texto, valid_token
      )
      for valid_token in vocab_valido
  }
  score = max(score.items(), key=operator.itemgetter(1)) if "jaro" in distancia else min(score.items(), key=operator.itemgetter(1))
  resultados[distancia] = {"most_likely_token": score[0] ,"score": score[1]}
resultados

{'jaro_similarity': {'most_likely_token': 'porqu',
  'score': 0.9444444444444445},
 'jaro_winkler_similarity': {'most_likely_token': 'porqu',
  'score': 0.9666666666666667},
 'damerau_levenshtein_distance': {'most_likely_token': 'porque', 'score': 1},
 'levenshtein_distance': {'most_likely_token': 'porque', 'score': 1}}

Os métodos de levenshtein conseguiram identificar a palavra correta! Enquanto os métodos de jaro acabaram achando `porqua` mais similar com `porqu`. Outro ponto interessante é que jaro_winkler_similarity deu uma pontuação maior do que somente jaro_similarity, visto que a maior parte da similaridade dos dois textos está no início `porqu`. Os métodos Damerau Levenshtein e Levenshtein deram o mesmo resultado, visto que não foi necessário nenhuma transposição para chegar de `porqua` para `porque`, somente a alteração de uma palavra, o `a` pelo `e`.

##### Usando a similaridade para limitar o vocabulário

Agora vamos usar a biblioteca rapidfuzz que já implementa a comparação de um texto com uma lista, extraindo o mais similar de acordo com a distância de Levenshtein e retornando a pontuação normalizada também. Com isso, podemos usá-la para limitar nosso vocabulário, alterando os textos com alta similaridade, enquanto os demais irão para um token chamado "desconhecido". Poderíamos ter implementado um loop usando a biblioteca jellyfish também.

O score de 80 foi um número arbitrário, podemos aumentar ou reduzir de acordo com o problema em questão.

In [43]:
from rapidfuzz import process
from rapidfuzz.distance import Levenshtein

items = list(vocab_tratamento.items())
itens_to_treat = dict(items[:20])

for key, value in itens_to_treat.items():
    new_token, score, _ = process.extractOne(key, vocab_valido.keys(), scorer=Levenshtein.normalized_similarity)
    replacement = new_token if score >= 0.80 else "desconhecido"
    df_reviews["cleaned_text_fuzzy"] = df_reviews["cleaned_text"].str.replace(f" {key} ", f" {replacement} ")
    print(f"Substituindo token {key} pelo token {replacement}")

Substituindo token estatisticos pelo token estatisticas
Substituindo token sigma pelo token siga
Substituindo token destravou pelo token desconhecido
Substituindo token steelcase pelo token desconhecido
Substituindo token jim pelo token desconhecido
Substituindo token andadores pelo token animadores
Substituindo token arrendondada pelo token arredondada
Substituindo token guste pelo token goste
Substituindo token calida pelo token calidad
Substituindo token actividades pelo token atividades
Substituindo token estudiar pelo token estudar
Substituindo token a16 pelo token desconhecido
Substituindo token derrotara pelo token derrotar
Substituindo token xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx pelo token xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
Substituindo token buddemeryer pelo token buddemeyer
Substituindo token inte

Podemos ver que grande parte das substituições fizeram bastante sentido, limpando os tokens para não ter números e letras teríamos um resultado ainda melhor. <br>
<br> Agora você já sabe como limpar os textos! Na próxima aula veremos como transformar os tokens em representações numéricas que irão alimentar nosso modelo.